# Evaluate the motion artifacts of BOLD data
This measure indexes the movement of the head from one volume to the next, and is calculated as the sum of the absolute values of the differentiated realignment estimates (by backwards differences) at every timepoint.
<br>
You need to adjust the threshould for number of affected volumes and threshould for absolut value for FD.
 
-----------------------------------------------------------
Script written by Mehdi Behroozi
<br>
Biopsychology, 
<br>
Ruhr-University Bochum, Bochum, Germany
<br>
(2022.03)

-----------------------------------------------------------

In [ ]:
import os
import glob
import shutil
from pathlib import Path

# Absolute threshold value for affected volume by motion
thresh = 0.9  # (edit here)

# Set data path
data_path = Path("/mnt/d/Data/Human/ObjectCategorization/analysis2")  # (edit here)

# Find bold files
bold_files = glob.glob(f"{data_path}/sub*/ses-02/func/*run*.nii.gz")

for curr_bold in bold_files:
    curr_bold_path = Path(curr_bold)
    print(f"Current data is: {curr_bold_path.name}")

    exp_name = curr_bold_path.stem.split("_")[-2]
    cur_dir = curr_bold_path.parent / exp_name
    print(cur_dir)

    # Create results directory if it doesn't exist
    if not cur_dir.exists():
        print("\t 1) Creating 'results' directory ...")
        cur_dir.mkdir(parents=True)
    else:
        print("\t 1) 'results' directory exists.")

    # Create motion_assess directory
    motion_assess_dir = cur_dir / "motion_assess"
    
    if not motion_assess_dir.exists():
        print(f"\t 2) Creating {motion_assess_dir} directory ...")
        motion_assess_dir.mkdir()
    else:
        print(f"\t 2) {motion_assess_dir} directory exists! Deleting old and creating new directory ...")
        shutil.rmtree(motion_assess_dir)
        motion_assess_dir.mkdir()

    # Running fsl_motion_outliers
    print(f"\t 3) Running fsl_motion_outliers with threshold of {thresh} ...\n")
    cmd = (
        f"fsl_motion_outliers -i {curr_bold} "
        f"-o {motion_assess_dir}/confound.txt --fd --thresh={thresh} "
        f"-s {motion_assess_dir}/fd_data.txt -p {motion_assess_dir}/fd_plot "
        f"-v > {motion_assess_dir}/outlier_output.txt"
    )
    os.system(cmd)

    # Create an empty placeholder file if confound.txt is missing
    confound_file = motion_assess_dir / "confound.txt"
    if not confound_file.exists():
        confound_file.touch()

# Plot framewise displasment results
The results for single subjects will be ploted

In [ ]:
%matplotlib inline

from IPython.display import display, Image
import matplotlib.pyplot as plt
import glob
from pathlib import Path

# Define data path
data_path = Path("/mnt/d/Data/Human/ObjectCategorization/analysis2")

# Find all fd_plot.png files
FD_files = list(data_path.glob("sub*/ses-02/func/run*/motion_assess/fd_plot.png"))

# Process each image file
if FD_files:
    for curr_FD in FD_files:
        curr_FD_path = Path(curr_FD)
        
        # Extract sub, session, func folder names dynamically
        sub = curr_FD_path.parts[-6]
        session = curr_FD_path.parts[-5]
        func = curr_FD_path.parts[-3]

        print(f"{sub}/{session}/{func}")

        # Show the plot
        plt.show()

        # Display the image
        display(Image(filename=str(curr_FD_path)))

else:
    print("No fd_plot.png files found.")

# Plot Histogram
This code will pull FD data from all subjects and plot the histogram

In [ ]:
# Import necessary libraries
import glob
import matplotlib.pyplot as plt
import os
from pathlib import Path

# Define data path
data_path = Path("/mnt/d/Data/Human/ObjectCategorization/analysis2")

# Find all fd_data.txt files
FD_files = list(data_path.glob("sub*/ses-02/func/run*/motion_assess/fd_data.txt"))

# Initialize list to store framewise displacement data
FD_data = []

# Read and process data from files
for cur_FD in FD_files:
    with open(cur_FD, "r") as f:
        FD_data.extend(float(line.strip()) for line in f if line.strip())  # Filters valid numeric values

# Plot histogram
plt.figure(figsize=(8, 6))
plt.hist(FD_data, bins=500, orientation="horizontal", color="skyblue", edgecolor="black")
plt.ylabel("Framewise displacement (mm)")
plt.xlabel("Volume count")

# Apply styling
plt.style.use("seaborn-bright")

# Optional: Save plots
# output_dir = data_path / "motion_parameters"
# output_dir.mkdir(exist_ok=True)  # Ensure output directory exists
# plt.savefig(output_dir / "FD.svg", format="svg", dpi=1200)
# plt.savefig(output_dir / "FD.png")

# Show the plot
plt.show()


# Finding number of affected volumes for each run

In [ ]:
import glob
import matplotlib.pyplot as plt
from pathlib import Path

# Define data path
data_path = Path("/mnt/d/Data/Human/ObjectCategorization/analysis2")

# Find all fd_data.txt files
FD_files = list(data_path.glob("sub*/ses-02/func/run*/motion_assess/fd_data.txt"))

# Define motion threshold
thresh = 0.9 # Absolute threshold value for affected volume by motion (edit here)

# Define function to compute length above threshold
def length_of_list(data, threshold):
    return sum(1 for value in data if value > threshold)

# Process each file
for cur_FD in FD_files:
    FD_data = []

    # Read and process data
    with open(cur_FD, "r") as f:
        FD_data = [float(line.strip()) for line in f if line.strip()]  # Removes empty lines and converts values

    # Extract relevant metadata from path
    cur_FD_path = Path(cur_FD)
    sub = cur_FD_path.parts[-6]
    session = cur_FD_path.parts[-5]
    motion_assess = cur_FD_path.parts[-3]

    # Print structured output
    print(f"{sub}/{session}/{motion_assess} \t --->> \t {length_of_list(FD_data, thresh)}")


In [ ]:
import glob
import os
import shutil
data_path = '/mnt/R/Data/Human/ObjectCategorization/analysis'
des= '/mnt/d/Data/Human/ObjectCategorization/analysis/motion'

FD_files = glob.glob('%s/sub*/ses-0*/func/*rest*/mc/bold_mcf.par'%(data_path))
for cur_FD in list(FD_files):
    print(cur_FD)
    # Construct the full path for the new file
    new_name = '_'.join(['mc', cur_FD.split('/')[-6],cur_FD.split('/')[-5],cur_FD.split('/')[-3]])
    new_file_path = os.path.join(des,new_name)

    # Copy the file to the new location with the new name
    shutil.copy(cur_FD, new_file_path)


In [ ]:
#Plot Histogram after excluding the affected subjects

In [ ]:
# Import necessary libraries
import glob
import matplotlib.pyplot as plt
import os
from pathlib import Path

# Define data path
data_path = Path("/mnt/d/Data/Human/ObjectCategorization/analysis2")

# Define subjects to exclude
excluded_subjects = {"sub-56", "sub-51", "sub-38", "sub-35", "sub-30", "sub-12"}  # Add subjects to exclude

# Find all fd_data.txt files
FD_files = list(data_path.glob("sub*/ses-02/func/*run*/motion_assess/fd_data.txt"))

# Initialize list to store framewise displacement data
FD_data = []

# Read and process data from files
for cur_FD in FD_files:
    cur_FD_path = Path(cur_FD)
    subject = cur_FD_path.parts[-6]  # Extracts 'sub-XX' from the path

    if subject in excluded_subjects:
        
        print(f"Skipping {subject}")
        continue  # Skip excluded subjects

    with open(cur_FD, "r") as f:
        FD_data.extend(float(line.strip()) for line in f if line.strip())  # Filters valid numeric values

# Plot histogram
plt.figure(figsize=(8, 6))
plt.hist(FD_data, bins=500, orientation="horizontal", color="skyblue", edgecolor="black")
plt.ylabel("Framewise displacement (mm)")
plt.xlabel("Volume count")

# Apply styling
plt.style.use("seaborn-bright")

# Optional: Save plots
output_dir = data_path / "motion_parameters"
output_dir.mkdir(exist_ok=True)  # Ensure output directory exists
plt.savefig(output_dir / "FD.svg", format="svg", dpi=1200)
plt.savefig(output_dir / "FD.png")

# Show the plot
plt.show()
